In [1]:
import os
import torch
import gym
import numpy as np
from TD3_torch.TD3 import TD3
from PIL import Image
from TD3_torch.utils import ReplayBuffer

In [2]:
env_name = 'BipedalWalkerHardcore-v2'
lr_base = 0.001
lr_decay = 0.0001
exp_noise_base = 0.2 
exp_noise_decay = 0.0001

random_seed = 234
gamma = 0.99                # discount for future rewards
batch_size = 1024        # num of transitions sampled from replay buffer
polyak = 0.9999              # target policy update parameter (1-tau)
policy_noise = 0.2          # target policy smoothing noise
noise_clip = 0.5
policy_delay = 2            # delayed policy updates parameter
max_episodes = 100000         # max num of episodes
max_timesteps = 3000        # max timesteps in one episode
max_buffer_length = 5000000
log_interval = 5           # print avg reward after interval


In [3]:
actor_config = [
        {'dim': [None, 256], 'dropout': False, 'activation': 'relu'},
        {'dim': [256, 256], 'dropout': True, 'activation':'relu'},
        {'dim': [256, 128], 'dropout': False, 'activation': 'relu'},       
        {'dim': [128, None],'dropout': False, 'activation': 'tanh'}
    ]
    
critic_config = [
        {'dim': [None, 512], 'dropout': False, 'activation': 'relu'},
        {'dim': [512, 512], 'dropout': False , 'activation':'relu'},
        {'dim': [512, 128], 'dropout': False, 'activation': 'relu'},       
        {'dim': [128, 1], 'dropout': False, 'activation': False},
    ]

In [4]:
class TD3Trainer():
    
    def __init__(self, env_name, actor_config, critic_config, random_seed=42, lr_base=0.001, lr_decay=0.00005, 
        exp_noise_base=0.3, exp_noise_decay=0.0001, gamma=0.99, batch_size=1024, 
        polyak=0.9999, policy_noise=0.2, noise_clip=0.5, policy_delay=2, 
        max_episodes=100000, max_timesteps=3000, max_buffer_length=5000000,
        log_interval=5, threshold=None, lr_minimum=1e-10, exp_noise_minimum=1e-10):        
        
        self.algorithm_name = 'td3_torch'
        self.env_name = env_name
        self.env = gym.make(env_name)
        self.state_dim = self.env.observation_space.shape[0]
        self.action_dim = self.env.action_space.shape[0]
        self.action_low = self.env.action_space.low
        self.action_high = self.env.action_space.high
        self.max_action = float(self.env.action_space.high[0])
        self.threshold = self.env.spec.reward_threshold
        
        self.actor_config = actor_config
        self.critic_config = critic_config
        self.actor_config[0]['dim'][0] = self.state_dim
        self.actor_config[-1]['dim'][1] = self.action_dim
        self.critic_config[0]['dim'][0] = self.state_dim + self.action_dim
        
        self.actor_config = actor_config
        self.critic_config = critic_config
        self.random_seed = random_seed
        self.lr_base = lr_base
        self.lr_decay = lr_decay   
        self.lr_minimum = lr_minimum
        self.exp_noise_base = exp_noise_base
        self.exp_noise_decay = exp_noise_decay     
        self.exp_noise_minimum = exp_noise_minimum                
        self.gamma = gamma
        self.batch_size = batch_size        
        self.polyak = polyak
        self.policy_noise = policy_noise
        self.noise_clip = noise_clip
        self.policy_delay = policy_delay
        self.max_episodes = max_episodes
        self.max_timesteps = max_timesteps
        self.max_buffer_length = max_buffer_length
        self.log_interval = log_interval
        
        self.directory = "./preTrained/{}/{}".format(self.algorithm_name, self.env_name) 
        self.filename = "{}_{}_{}".format(self.algorithm_name, self.env_name, self.random_seed)
        
        self.policy = TD3(self.actor_config, self.critic_config, self.max_action)   
        self.replay_buffer = ReplayBuffer(max_length=self.max_buffer_length)
        
        self.reward_history = []
        self.make_plots = False       
        
        if self.random_seed:
            print("Random Seed: {}".format(self.random_seed))
            self.env.seed(self.random_seed)
            torch.manual_seed(self.random_seed)
            np.random.seed(self.random_seed)
        
    def train(self):
        
        print("action_space={}".format(self.env.action_space))
        print("obs_space={}".format(self.env.observation_space))
        print("threshold={} \n".format(self.threshold))     

        # loading models
        self.policy.load(self.directory, self.filename)
                
        # logging variables:        
        log_f = open("train_{}.txt".format(self.algorithm_name), "w+")

        # training procedure:
        for episode in range(1, self.max_episodes+1):
            ep_reward = 0
            state = self.env.reset()
            
            # calculate params
            exploration_noise = max(self.exp_noise_base / (1.0 + episode * self.exp_noise_decay), self.exp_noise_minimum)
            learning_rate = max(self.lr_base / (1.0 + episode * self.lr_decay), self.lr_minimum)            
            self.policy.set_optimizers(lr=learning_rate)

            for t in range(self.max_timesteps):
                
                # select action and add exploration noise:
                action = self.policy.select_action(state)
                action = action + np.random.normal(0, exploration_noise, size=self.action_dim)
                action = action.clip(self.action_low, self.action_high)

                # take action in env:
                next_state, reward, done, _ = self.env.step(action)
                self.replay_buffer.add((state, action, reward, next_state, float(done)))
                state = next_state

                ep_reward += reward

                # if episode is done then update policy:
                if done or t==(self.max_timesteps-1):
                    self.policy.update(self.replay_buffer, t, self.batch_size, self.gamma, self.polyak, 
                                       self.policy_noise, self.noise_clip, self.policy_delay)
                    break

            self.reward_history.append(ep_reward)
            avg_reward = np.mean(self.reward_history[-100:]) 

            # logging updates:        
            log_f.write('{},{}\n'.format(episode, ep_reward))
            log_f.flush()
            
            # Calculate polyak
            #part = (env.spec.reward_threshold - avg_reward) / (env.spec.reward_threshold + 150)
            #if part > 1:
            #    part = 1
            #polyak = polyak_int[0] + (1 - part) * (polyak_int[1] - polyak_int[0])     

            # Calculate LR
            #part = min((env.spec.reward_threshold - avg_reward) / (env.spec.reward_threshold + 150), 1)
                        
            avg_actor_loss = np.mean(self.policy.actor_loss_list[-100:])
            avg_Q1_loss = np.mean(self.policy.Q1_loss_list[-100:])
            avg_Q2_loss = np.mean(self.policy.Q2_loss_list[-100:])

            if not self.make_plots and len(self.policy.actor_loss_list) > 200:
                self.policy.actor_loss_list.pop(0)
                self.policy.Q1_loss_list.pop(0)
                self.policy.Q2_loss_list.pop(0)  
                self.reward_history.pop(0)    

            # Print avg reward every log interval:
            if episode % self.log_interval == 0:            
                self.policy.save(self.directory, self.filename)
                print("Ep:{}   Rew:{:3.2f}  Avg Rew:{:3.2f}  LR:{:8.8f}   Polyak:{:5.5f}  Bf:{:2.0f}  EN:{:0.4f}  Loss: {:5.3f} {:5.3f} {:5.3f}".format(
                    episode, ep_reward, avg_reward, learning_rate, self.polyak, self.replay_buffer.get_fill(), 
                    exploration_noise, avg_actor_loss, avg_Q1_loss, avg_Q2_loss))
                
            # if avg reward > threshold then save and stop traning:
            if avg_reward >= self.threshold: 
                print("Ep:{}   Rew:{:3.2f}  Avg Rew:{:3.2f}  LR:{:8.8f}   Polyak:{:5.5f}  Bf:{:2.0f}  EN:{:0.4f}  Loss: {:5.3f} {:5.3f} {:5.3f}".format(
                    episode, ep_reward, avg_reward, learning_rate, self.polyak, self.replay_buffer.get_fill(), 
                    exploration_noise, avg_actor_loss, avg_Q1_loss, avg_Q2_loss))
                print("########## Solved! ###########")
                name = self.filename + '_solved'
                self.policy.save(self.directory, name)
                log_f.close()
                break    
       
    def test(self, episodes=3, render=True, save_gif=True):              

        for episode in range(1, self.episodes+1):
            ep_reward = 0
            state = self.env.reset()
            
            for t in range(self.max_timesteps):
                action = self.policy.select_action(state)
                state, reward, done, _ = self.env.step(action)
                ep_reward += reward
                if render:
                    env.render()
                    if save_gif:
                        dirname = './gif/{}/{}'.format(self.algorithm_name, episode)
                        if not os.path.isdir(dirname):
                            os.mkdir(dirname)
                        img = self.env.render(mode = 'rgb_array')
                        img = Image.fromarray(img)
                        img.save('./gif/{}/{}/{}.jpg'.format(self.algorithm_name, episode, t))
                if done:
                    break

            print('Test episode: {}\tReward: {}'.format(episode, int(ep_reward)))
            ep_reward = 0
            self.env.close()        
            

In [5]:
agent = TD3Trainer(env_name, actor_config, critic_config, random_seed=random_seed, lr_base=lr_base, lr_decay=lr_decay, 
                   exp_noise_base=exp_noise_base, exp_noise_decay=exp_noise_decay, gamma=gamma, batch_size=batch_size,
                   polyak=polyak, policy_noise=policy_noise, noise_clip=noise_clip, policy_delay=policy_delay, 
                   max_episodes=max_episodes, max_timesteps=max_timesteps, max_buffer_length=max_buffer_length, 
                   log_interval=log_interval)
agent.train()

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
ACTOR=Sequential(
  (0): Linear(in_features=24, out_features=256, bias=True)
  (1): ReLU()
  (2): Linear(in_features=256, out_features=256, bias=True)
  (3): Dropout(p=0.2)
  (4): ReLU()
  (5): Linear(in_features=256, out_features=128, bias=True)
  (6): ReLU()
  (7): Linear(in_features=128, out_features=4, bias=True)
  (8): Tanh()
)


RuntimeError: CUDA error: out of memory

In [ ]:
agent.test()